In [1]:
from dotenv import load_dotenv
from langgraph.graph import StateGraph,END,add_messages,START
from langgraph.types import Command,interrupt
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from typing import TypedDict,Annotated
from langchain_community.vectorstores import Chroma
load_dotenv()
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

/var/folders/gk/cm66kf9j5qj4vzctc4gt_sxr0000gn/T/ipykernel_46744/121061764.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [4]:
from langchain.schema import Document

# NexaTech Solutions - Company Knowledge Base Documents
docs = [
    Document(
        page_content=""" 
NexaTech Solutions is a leading technology consulting and software development company founded in 2018.
We specialize in enterprise software solutions, cloud infrastructure, and digital transformation services.
Our headquarters is located in Austin, Texas, with additional offices in Denver, Seattle, and Miami.
The company employs over 250 skilled professionals including software engineers, data scientists, and project managers.
We serve clients across healthcare, finance, retail, and manufacturing industries.
Our core values include innovation, reliability, and customer success.
Office Hours: Monday-Friday 8:00 AM - 6:00 PM (Central Time)
Main Reception: (512) 555-0100
Address: 4567 Tech Boulevard, Austin, TX 78701
CEO: Sarah Mitchell | CTO: David Chen
        """,
        metadata={"source": "nexatech/company_overview", "department": "administration", "last_updated": "2024-01-15"}
    ),
    
    Document(
        page_content=""" 
NexaTech Solutions Software Development Department is our largest division with 80+ experienced developers.
We develop custom web applications, mobile apps, and enterprise software using modern technologies.
Our tech stack includes React, Node.js, Python, Java, AWS, and Kubernetes.
The development team follows Agile methodologies with 2-week sprints and continuous integration/deployment.
We offer full-stack development, API integration, database design, and system architecture services.
Current projects include a healthcare management system, e-commerce platform, and IoT dashboard.
Development Hours: Monday-Friday 9:00 AM - 6:00 PM, On-call support available 24/7
Team Lead: Michael Rodriguez | Email: dev@nexatech.com
Emergency Support: (512) 555-0101
Project Timeline: Average 3-6 months for enterprise solutions
        """,
        metadata={"source": "nexatech/software_development", "department": "development", "last_updated": "2024-01-10"}
    ),
    
    Document(
        page_content=""" 
NexaTech Solutions Cloud Infrastructure Division manages enterprise cloud deployments and DevOps operations.
We provide AWS, Azure, and Google Cloud consulting, migration services, and 24/7 infrastructure monitoring.
Our services include containerization, microservices architecture, and automated CI/CD pipelines.
The team maintains 99.9% uptime for client applications and provides disaster recovery solutions.
We specialize in cost optimization, security compliance, and scalable infrastructure design.
Current certifications include AWS Solutions Architect, Azure DevOps Engineer, and Google Cloud Professional.
Operations Center Hours: 24/7 monitoring and support
Response Time: Critical issues within 15 minutes, Standard issues within 2 hours
Team Lead: Jennifer Park | Email: cloud@nexatech.com
Emergency Hotline: (512) 555-0102
Monthly SLA Reports: Available first Monday of each month
        """,
        metadata={"source": "nexatech/cloud_infrastructure", "department": "operations", "last_updated": "2024-01-08"}
    ),
    
    Document(
        page_content="""
NexaTech Solutions Data Analytics Department transforms raw data into actionable business insights.
Our team of 15 data scientists and analysts work with machine learning, predictive modeling, and business intelligence.
We use tools like Python, R, Tableau, Power BI, and Apache Spark for data processing and visualization.
Services include data warehousing, ETL pipeline development, and custom dashboard creation.
Recent projects include customer churn prediction, inventory optimization, and fraud detection systems.
The department maintains partnerships with leading data platforms and cloud providers.
Analytics Hours: Monday-Friday 8:30 AM - 5:30 PM (Central Time)
Consultation Availability: Tuesday-Thursday 2:00 PM - 4:00 PM
Team Lead: Dr. Amanda Foster | Email: analytics@nexatech.com
Phone: (512) 555-0103
Report Delivery: Standard reports within 5 business days
""",
        metadata={"source": "nexatech/data_analytics", "department": "analytics", "last_updated": "2024-01-12"}
    ),
    
    Document(
        page_content="""
NexaTech Solutions Cybersecurity Division provides comprehensive security services and compliance solutions.
Our certified security experts conduct penetration testing, vulnerability assessments, and security audits.
We implement multi-factor authentication, encryption protocols, and intrusion detection systems.
The team ensures compliance with GDPR, HIPAA, SOC 2, and other regulatory requirements.
Services include security training, incident response, and 24/7 threat monitoring.
We maintain a Security Operations Center (SOC) with advanced threat intelligence capabilities.
Security Operations: 24/7 monitoring and incident response
Business Hours: Monday-Friday 8:00 AM - 7:00 PM (Central Time)
Team Lead: Robert Thompson | Email: security@nexatech.com
Emergency Security Line: (512) 555-0911
Response Time: Security incidents within 30 minutes
""",
        metadata={"source": "nexatech/cybersecurity", "department": "security", "last_updated": "2024-01-14"}
    ),
    
    Document(
        page_content="""
NexaTech Solutions Customer Success Department ensures client satisfaction and long-term partnerships.
Our team of 12 customer success managers provides onboarding, training, and ongoing support services.
We offer comprehensive documentation, video tutorials, and regular check-in meetings with clients.
The department tracks customer health scores, usage metrics, and satisfaction surveys.
Services include project management, user training, and technical support coordination.
We maintain a 95% client retention rate and 4.8/5 customer satisfaction score.
Support Hours: Monday-Friday 7:00 AM - 8:00 PM (Central Time)
Weekend Support: Saturday 9:00 AM - 2:00 PM (emergency only)
Team Lead: Lisa Martinez | Email: success@nexatech.com
Support Ticket System: support.nexatech.com
Phone Support: (512) 555-0104
""",
        metadata={"source": "nexatech/customer_success", "department": "support", "last_updated": "2024-01-11"}
    ),
    
    Document(
        page_content="""
NexaTech Solutions Human Resources Department manages talent acquisition, employee development, and company culture.
We employ 250+ professionals across four office locations with plans to expand to 350 by 2024.
Our benefits package includes health insurance, retirement plans, flexible work arrangements, and professional development funds.
The department organizes quarterly team building events, annual conferences, and skills training programs.
We maintain partnerships with local universities for internship and graduate recruitment programs.
Current open positions include Senior Software Engineers, Data Scientists, and Project Managers.
HR Office Hours: Monday-Friday 8:00 AM - 5:00 PM (Central Time)
Recruitment Hours: Monday-Thursday 9:00 AM - 4:00 PM
HR Director: Patricia Wilson | Email: hr@nexatech.com
Phone: (512) 555-0105
Career Portal: careers.nexatech.com
""",
        metadata={"source": "nexatech/human_resources", "department": "hr", "last_updated": "2024-01-13"}
    ),
    
    Document(
        page_content="""
NexaTech Solutions Finance and Accounting Department manages financial operations, budgeting, and client billing.
We process over $25 million in annual revenue and maintain detailed financial reporting for stakeholders.
The department handles accounts receivable, payroll processing, and vendor management.
We provide quarterly financial reports, project cost analysis, and budget forecasting services.
Our accounting practices comply with GAAP standards and undergo annual independent audits.
We offer flexible payment terms and multiple billing options for enterprise clients.
Finance Hours: Monday-Friday 8:00 AM - 6:00 PM (Central Time)
Billing Inquiries: Monday-Friday 9:00 AM - 5:00 PM
CFO: Mark Stevens | Email: finance@nexatech.com
Accounts Receivable: (512) 555-0106
Payment Portal: billing.nexatech.com
Invoice Questions: billing@nexatech.com
""",
        metadata={"source": "nexatech/finance_accounting", "department": "finance", "last_updated": "2024-01-09"}
    ),
]

# Description of the NexaTech Solutions Retrieval System
retrieval_system_description = """
This document retrieval system serves as the central knowledge base for NexaTech Solutions, a full-service technology consulting and software development company based in Austin, Texas. The system contains comprehensive information about all company departments including software development, cloud infrastructure, data analytics, cybersecurity, customer success, human resources, and finance. It provides real-time access to operational hours, contact details, service offerings, team leadership information, and emergency support numbers across all divisions. The system enables semantic search capabilities allowing employees, clients, and partners to quickly find relevant information about services, contacts, schedules, and procedures using natural language queries.
"""
db = Chroma.from_documents(documents=docs,embedding=embeddings)

In [ ]:
retriever = db.as_retriever(search_type="mmr",search_kwargs={"k":4})
retriever.invoke("what are tinings of recruitment hours")

[Document(metadata={'source': 'nexatech/human_resources', 'department': 'hr', 'last_updated': '2024-01-13'}, page_content='\nNexaTech Solutions Human Resources Department manages talent acquisition, employee development, and company culture.\nWe employ 250+ professionals across four office locations with plans to expand to 350 by 2024.\nOur benefits package includes health insurance, retirement plans, flexible work arrangements, and professional development funds.\nThe department organizes quarterly team building events, annual conferences, and skills training programs.\nWe maintain partnerships with local universities for internship and graduate recruitment programs.\nCurrent open positions include Senior Software Engineers, Data Scientists, and Project Managers.\nHR Office Hours: Monday-Friday 8:00 AM - 5:00 PM (Central Time)\nRecruitment Hours: Monday-Thursday 9:00 AM - 4:00 PM\nHR Director: Patricia Wilson | Email: hr@nexatech.com\nPhone: (512) 555-0105\nCareer Portal: careers.nexa

In [29]:
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool
retrieval_tool = create_retriever_tool(
    retriever,
    "retriever_tool",
    retrieval_system_description
)

@tool
def off_topic():
    """catch this tool If the question is not related to this context: 
      NexaTech Solutions, a full-service technology consulting and software development company based in Austin, Texas. The system contains comprehensive information about all company departments including software development, cloud infrastructure, data analytics, 
      cybersecurity, customer success, human resources, and finance. It provides real-time access to operational hours, contact details,
        service offerings, team leadership information, and emergency support numbers across all divisions. 
        The system enables semantic search capabilities allowing employees, clients, and partners to quickly find relevant 
        information about services, contacts, schedules, and procedures using natural language queries.
"""
    return "Forbidden - Not in my boundaries"
llm=llm.bind_tools([retrieval_tool,off_topic])

In [ ]:
from langgraph.graph import StateGraph,END,add_messages,START
from langchain_core.messages import BaseMessage
from typing import TypedDict,Annotated,Sequence
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]

def llm_node(state: AgentState):
    response = llm.invoke(state["messages"])
    return {
        "messages":[response]
    }

def tool_router(state:AgentState):
    message = state["messages"][-1]
    if message.tool_call:
        return "tool_exec"
    else:
        return END


In [40]:
from langgraph.prebuilt import ToolNode
graph = StateGraph(AgentState)

graph.add_node("llm_node",llm_node)


tool_node = ToolNode([off_topic,retrieval_tool])

graph.add_node("tool_node",tool_node)

graph.add_edge("tool_node","llm_node")

graph.add_conditional_edges(
    "llm_node",tool_router,{"tool_exec":"tool_node",END:END}
)
graph.add_edge(START,"llm_node")

app=graph.compile()

In [ ]:
app.invoke("messages":[HumanMessage(content="what are the timings for research")]})

ValueError: Invalid input type <class 'langchain_core.messages.human.HumanMessage'>. Must be a PromptValue, str, or list of BaseMessages.